In [1]:
import pandas as pd
import numpy as np
import camelot
import warnings
warnings.filterwarnings('ignore')

# PDF 1

In [2]:
virginia1 = camelot.read_pdf('virginia1.pdf')
virginia1_df = virginia1[0].df

In [3]:
virginia1_df["1_copy"] = virginia1_df[1]
column_order = [0, 1] + ["1_copy"] + list(range(2, 15))
virginia1_df = virginia1_df[column_order]
virginia1_df.columns = np.arange(0, 16)

# remove \n in the first three rows of the table
for i in np.arange(3):
    virginia1_df.iloc[i] = virginia1_df.iloc[i].str.replace('\n', '')

virginia1_df[0] = virginia1_df[0].str.replace("\n", '')

In [4]:
def scrape_text(text):
    text_sequence = text.split("\n")
    if len(text_sequence) == 1:
        return text_sequence[0]
    else:
        return text_sequence[1]

virginia1_df[2] = virginia1_df[2].apply(scrape_text)

In [5]:
# Iterate over the series elements
def fix_rows(row):
    for idx, val in row.items():
        parts = val.split('\n')
        if len(parts) > 1:
            # Assign the first part
            row[idx] = parts[0].strip()  
            # Assign the second part to the next index
            row[idx + 1] = parts[1].strip() 
    return row

for i in np.arange(4, virginia1_df.shape[0]):
    cleaned_row = fix_rows(virginia1_df.iloc[i])
    virginia1_df.iloc[i] = cleaned_row

# manually adjust edge cases
virginia1_df.iloc[3, 1] = "Finished"
virginia1_df.iloc[10, 14] = 54.0

# adjust the null spaces in the first two rows
virginia1_df.iloc[0] = virginia1_df.iloc[0].replace('', None).fillna(method = 'ffill')
virginia1_df.iloc[1] = virginia1_df.iloc[1].replace('', None).fillna(method = 'ffill')

# set columns 
virginia1_df.columns = virginia1_df.iloc[0]
virginia1_df = virginia1_df.drop(virginia1_df.index[0])

In [7]:
virginia1_df.to_csv("virginia1.csv")

# PDF 2

In [34]:
virginia2_tables = camelot.read_pdf('virginia2.pdf', pages = '1,2,3')

In [35]:
def clean_table(table, start_col, end_col):
    # replace "\n"
    for i in np.arange(4):
        table.iloc[i] = table.iloc[i].str.replace('\n', ' ')
    
    # reformat the columns
    for i in np.arange(start_col, end_col + 1):
        # grab the current column
        split_values = table.iloc[4, i].split('\n')
        for idx, val in enumerate(split_values):
            table.iloc[(4+idx), i] = val

In [36]:
virginia2 = virginia2_tables[0].df
clean_table(virginia2, 1, 8)
virginia2.to_csv("virginia2.csv")

In [37]:
virginia3 = virginia2_tables[1].df
clean_table(virginia3, 1, 8)
virginia3.to_csv("virginia3.csv")

In [38]:
virginia4 = virginia2_tables[2].df
clean_table(virginia4, 1, 8)
virginia4.to_csv("virginia4.csv")